In [ ]:
from pandas import DataFrame
from DATA225utils import make_connection, dataframe_query

In [ ]:
conn_db = make_connection(config_file = 'bossbunch_db.ini')
cursor_db = conn_db.cursor()

In [ ]:
conn_warehouse = make_connection(config_file = 'bossbunch_wh.ini')
cursor_warehouse = conn_warehouse.cursor()

In [ ]:
def make_table(table, sql):
    cursor_warehouse.execute(f"DROP TABLE IF EXISTS {table}")
    cursor_warehouse.execute(sql)

In [ ]:
def display_table(table, order_by=''):
    sql = f"SELECT * FROM {table}"
    
    if order_by != '':
        sql = sql + " ORDER BY " + order_by
        
    _, df = dataframe_query(conn_warehouse, sql)    
    return df

In [ ]:
drop_tables_if_exists(['feedback'])
drop_tables_if_exists(['Wishlist'])
drop_tables_if_exists(['Calendar'])
drop_tables_if_exists(['Wine'])
drop_tables_if_exists(['SalesPerWOC'])

## Feedback Dimesnsion Table

In [ ]:
sql = ( """
        CREATE TABLE Feedback
        (
            FeedbackKey INT NOT NULL AUTO_INCREMENT,
            Winename VARCHAR(50),
            Rating INT,
            PurchaseExperience INT
            PRIMARY KEY(FeedbackKey)
        );
        """
      )

make_table('Feedback', sql)

In [ ]:
sql = ( """
        INSERT INTO bossbunch_wh.Feedback(Winename, 
                                            Rating, PurchaseExperience
                                          )
            SELECT bossbunch_db.Feedback.Winename, 
                   bossbunch_db.Feedback.Rating, 
                   bossbunch_db.Feedback.PurchaseExperience
            FROM bossbunch_db.Feedback
            
        """
      )

cursor_db.execute(sql)
conn_db.commit()

## Wishlist Dimesnsion Table

In [ ]:
sql = ( """
        CREATE TABLE Wishlist
        (
            WishlistKey INT NOT NULL AUTO_INCREMENT,
            WishlistID INT,
            CustomerID INT,
            WineID VARCHAR(25),
            AddedDate DATETIME DEFAULT CURRENT_TIMESTAMP, 
            PRIMARY KEY(WishlistKey)
        )
        """
      )

make_table('Wishlist', sql)

In [ ]:
sql = ( """
        INSERT INTO bossbunch_wh.Wishlist(WishlistID, 
                                            CustomerID, WineID, AddedDate
                                          )
            SELECT bossbunch_db.Wishlist.WishlistID, 
                   bossbunch_db.Wishlist.CustomerID, 
                   bossbunch_db.Wishlist.WineID,
                   bossbunch_db.Wishlist.AddedDate
            FROM bossbunch_db.Wishlist
            
        """
      )

cursor_db.execute(sql)
conn_db.commit()

## Calendar Dimesnion Table

In [ ]:
sql = ( """
        CREATE TABLE bossbunch_wh.Calendar (
        CalendarKey INT NOT NULL AUTO_INCREMENT,
        FullDate DATE,
        DayOfMonth INT,
        DayOfWeek VARCHAR(55),
        CustomTimestamp TIMESTAMP,
        Month VARCHAR(55),
        Quarter VARCHAR(55),
        Year VARCHAR(55),
        PRIMARY KEY(CalendarKey)
        );
        """
      )

make_table('Calendar', sql)

In [ ]:
sql = ( """
        INSERT INTO bossbunch_wh.Calendar(FullDate, DayOfMonth, DayOfWeek, CustomTimestamp, Month, Quarter, Year)
            SELECT DISTINCT Date AS FullDate,DAY(Date) AS DayOfMonth, DAYNAME(Date) AS DayOfWeek,
                            Date AS CustomTimestamp, MONTHname(Date) AS Month,
                            CONCAT('Q',QUARTER(Date)) AS Quarter, YEAR(Date) AS Year
            FROM bossbunch_db.Orders;
        """
      )

cursor_warehouse.execute(sql)
conn_warehouse.commit()

## Wine Dimension Table

In [ ]:
sql = ( """
        CREATE TABLE bossbunch_wh.Wine (
        WineKey INT NOT NULL AUTO_INCREMENT,
        WineID VARCHAR(10),
        WineName VARCHAR(255),
        WineType VARCHAR(255),
        WineGrapeVariety VARCHAR(255),
        WineVintage VARCHAR(255),
        WinePricePerUnit NUMERIC(5,2),
        PRIMARY KEY(WineKey)
        );
        """
      )

make_table('Wine', sql)

In [ ]:
sql = ( """
        INSERT INTO bossbunch_wh.Wine (WineID, WineName, WineType, WineGrapeVariety, WineVintage, WinePricePerUnit)
        SELECT
            w.WineID,
            w.WineName,
            wt.WineType,
            gv.Variety,
            w.Vintage,
            w.Price
        FROM
            bossbunch_db.Wine w
            JOIN bossbunch_db.WineType wt ON w.WineTypeID = wt.WineTypeID
            JOIN bossbunch_db.GrapeVariety gv ON w.VarietyID = gv.VarietyID;

        """
      )

cursor_warehouse.execute(sql)
conn_warehouse.commit()

## SalesPerWOC Fact Table

In [ ]:
sql = ( """
        CREATE TABLE bossbunch_wh.SalesPerWOC (
          CalendarKey INT,
          FOREIGN KEY fk_calendar(CalendarKey) REFERENCES Calendar(CalendarKey),
          WineKey INT,
          FOREIGN KEY fk_wine(WineKey) REFERENCES Wine(WineKey),
          OrdersKey INT,
          FOREIGN KEY fk_orders(OrdersKey) REFERENCES Orders(OrdersKey),
          RevenueInDollars NUMERIC(10,2),
          UnitsSold INT,
          CONSTRAINT pk_sales PRIMARY KEY (CalendarKey, WineKey, OrdersKey)
        );
        """
      )

make_table('SalesPerWOC', sql)

In [ ]:
sql = ("""
    INSERT IGNORE INTO bossbunch_wh.SalesPerWOC (CalendarKey, WineKey, OrdersKey, RevenueInDollars, UnitsSold)
    SELECT
        bossbunch_wh.Calendar.CalendarKey,
        bossbunch_wh.Wine.WineKey,
        bossbunch_wh.Orders.OrdersKey,
        bossbunch_wh.Wine.WinePricePerUnit * bossbunch_db.Items.quantity AS RevenueInDollars,
        bossbunch_db.Items.Quantity AS UnitsSold
    FROM
        bossbunch_wh.Calendar
        INNER JOIN bossbunch_wh.Orders ON bossbunch_wh.Orders.Date = bossbunch_wh.Calendar.FullDate 
        INNER JOIN bossbunch_db.Items ON bossbunch_db.Items.OrderId = bossbunch_wh.Orders.OrderId
        INNER JOIN bossbunch_wh.Wine ON bossbunch_wh.Wine.WineID = bossbunch_db.Items.ItemID
    WHERE NOT EXISTS (
        SELECT 1 FROM bossbunch_wh.SalesPerWOC
        WHERE bossbunch_wh.SalesPerWOC.CalendarKey = bossbunch_wh.Calendar.CalendarKey
        AND bossbunch_wh.SalesPerWOC.WineKey = bossbunch_wh.Wine.WineKey
        AND bossbunch_wh.SalesPerWOC.OrdersKey = bossbunch_wh.Orders.OrdersKey
        
    )
""")

cursor_warehouse.execute(sql)
conn_warehouse.commit()